In [1]:
from scipy.optimize import minimize

loading data from the sql 

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# --- PostgreSQL Connection ---
engine = create_engine('postgresql://akilfiros:@127.0.0.1:5432/postgres')

# --- Load historical price data ---
query = "SELECT date, ticker, close FROM financial_data"
df = pd.read_sql(query, engine)
df['date'] = pd.to_datetime(df['date'])

# --- Pivot to get price matrix [date x ticker] ---
price_df = df.pivot(index='date', columns='ticker', values='close').sort_index()
price_df = price_df.dropna(axis=1)  # Drop tickers with missing data

# --- Compute log returns ---
log_returns = np.log(price_df / price_df.shift(1)).dropna()

# ✅ Preview
print("Log returns shape:", log_returns.shape)
print(log_returns.head())


Log returns shape: (1342, 92)
ticker          AAPL      ADBE       ADI       ADP       AEP      ALGN  \
date                                                                     
2020-01-03 -0.009769 -0.007865 -0.017760 -0.002116 -0.001071 -0.011487   
2020-01-06  0.007937  0.005710 -0.011818  0.001352  0.003315  0.019212   
2020-01-07 -0.004714 -0.000959  0.022496 -0.012178  0.000214 -0.009913   
2020-01-08  0.015958  0.013348  0.008991  0.009354 -0.002993  0.010333   
2020-01-09  0.021018  0.007607  0.000000  0.008800  0.005019  0.036190   

ticker          AMAT       AMD      AMGN      AMZN  ...      TEAM      TSLA  \
date                                                ...                       
2020-01-03 -0.016045 -0.010236 -0.006812 -0.012213  ...  0.001301  0.029203   
2020-01-06 -0.021801 -0.004330  0.007645  0.014776  ...  0.037096  0.019072   
2020-01-07  0.028477 -0.002897 -0.009450  0.002089  ...  0.003596  0.038067   
2020-01-08 -0.000649 -0.008743  0.000756 -0.007839  ... 

calculating expected returns & covariance matirx 

In [3]:
# Assuming 252 trading days per year
expected_returns = log_returns.mean() * 252

# --- Compute annualized covariance matrix ---
cov_matrix = log_returns.cov() * 252

# --- Preview ---
print("✅ Expected Returns (annualized):")
print(expected_returns.head())

print("\n✅ Covariance Matrix (annualized):")
print(cov_matrix.iloc[:5, :5])  # Show a 5x5 slice


✅ Expected Returns (annualized):
ticker
AAPL    0.188583
ADBE    0.025454
ADI     0.109049
ADP     0.129259
AEP     0.061917
dtype: float64

✅ Covariance Matrix (annualized):
ticker      AAPL      ADBE       ADI       ADP       AEP
ticker                                                  
AAPL    0.107145  0.078212  0.074367  0.049137  0.026762
ADBE    0.078212  0.153232  0.081179  0.052999  0.020500
ADI     0.074367  0.081179  0.136338  0.055804  0.022214
ADP     0.049137  0.052999  0.055804  0.076025  0.033876
AEP     0.026762  0.020500  0.022214  0.033876  0.060987


In [4]:
# --- Prepare data ---
tickers = expected_returns.index.tolist()
mu = expected_returns.values
cov = cov_matrix.values
n_assets = len(tickers)

# --- Objective function: portfolio variance ---
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# --- Constraints: fully invested ---
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

# --- Bounds: long-only portfolio (0 <= w <= 1) ---
bounds = [(0.0, 1.0) for _ in range(n_assets)]

# --- Initial guess: equal weighting ---
w0 = np.ones(n_assets) / n_assets

# --- Solve optimization ---
result = minimize(portfolio_variance, w0, args=(cov,), method='SLSQP',
                  bounds=bounds, constraints=constraints)

# --- Extract optimal weights ---
optimal_weights = pd.Series(result.x, index=tickers)

# --- Display result ---
print("✅ Optimal Weights (Minimum Variance Portfolio):")
print(optimal_weights[optimal_weights > 0].sort_values(ascending=False))


✅ Optimal Weights (Minimum Variance Portfolio):
KDP     1.792750e-01
GILD    1.400231e-01
AEP     1.286571e-01
AZN     1.027372e-01
COST    1.010828e-01
EA      9.360043e-02
PCAR    5.675562e-02
REGN    3.058151e-02
MAR     3.016993e-02
NTES    2.292824e-02
TTWO    2.291728e-02
MDLZ    2.273541e-02
KHC     2.021772e-02
TCOM    1.759886e-02
ORLY    1.574071e-02
DLTR    9.147910e-03
PANW    3.991462e-03
AMGN    1.839696e-03
MCHP    3.634374e-17
LRCX    3.454401e-17
GOOG    2.637643e-17
ADP     2.540928e-17
ADI     2.232876e-17
CSX     2.149076e-17
CDW     2.142107e-17
ILMN    1.636428e-17
AVGO    1.493871e-17
AMAT    1.413364e-17
CTAS    1.328084e-17
AMD     1.316125e-17
BIIB    1.256055e-17
ANSS    1.229979e-17
MSTR    1.110168e-17
MRVL    1.083597e-17
SBUX    1.000519e-17
MELI    9.637010e-18
MSFT    9.186955e-18
MTCH    8.466155e-18
SNPS    6.410464e-18
AXON    5.688195e-18
VRTX    5.141235e-18
CDNS    4.386370e-18
META    4.219155e-18
DOCU    4.051757e-18
TEAM    2.687576e-18
BKNG   


- These are the optimal asset weights (allocations) that minimize the total portfolio variance, assuming:
    - Full investment (sum = 1)
    - No short selling (weights ≥ 0)
- Assets with very small values (e.g., 1e-17) are essentially excluded from the portfolio — their weight is effectively zero.

Calculating for Constrained Portfolio Optimization (Target Return + Max Weight)

In [5]:
# --- Target return: e.g., at least 5% annual return ---
target_return = 0.05

# --- Objective: minimize portfolio variance ---
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# --- Constraints: fully invested + target return ---
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # weights sum to 1
    {'type': 'ineq', 'fun': lambda w: w @ mu - target_return}  # return constraint
]

# --- Bounds: long-only + max 10% in any asset ---
max_weight = 0.10
bounds = [(0.0, max_weight) for _ in range(n_assets)]

# --- Initial guess ---
w0 = np.ones(n_assets) / n_assets

# --- Solve ---
result_constrained = minimize(portfolio_variance, w0, args=(cov,), method='SLSQP',
                              bounds=bounds, constraints=constraints)

# --- Extract optimal weights ---
optimal_weights_constrained = pd.Series(result_constrained.x, index=tickers)

# --- Display non-zero weights ---
print("✅ Constrained Optimal Weights:")
print(optimal_weights_constrained[optimal_weights_constrained > 0].sort_values(ascending=False))

✅ Constrained Optimal Weights:
GILD     1.000000e-01
AEP      1.000000e-01
EA       1.000000e-01
KDP      1.000000e-01
COST     1.000000e-01
AZN      1.000000e-01
PCAR     6.624417e-02
MDLZ     5.976755e-02
REGN     4.738069e-02
KHC      4.419240e-02
MAR      3.335441e-02
ORLY     3.006236e-02
NTES     2.788278e-02
TTWO     2.481642e-02
AMGN     2.246658e-02
TCOM     1.831134e-02
DLTR     1.390944e-02
XEL      6.549449e-03
PANW     3.646597e-03
WBA      1.415808e-03
PEP      4.114538e-17
BIDU     3.025125e-17
QCOM     2.580810e-17
MU       2.099247e-17
KLAC     1.977335e-17
CTAS     1.948191e-17
FAST     1.854118e-17
OKTA     1.726911e-17
AVGO     1.717654e-17
CHTR     1.662152e-17
SBUX     1.601021e-17
VRSK     1.590026e-17
ON       1.570523e-17
CDW      1.506206e-17
INTC     1.447309e-17
BIIB     1.268534e-17
LULU     1.161516e-17
PDD      1.032470e-17
MNST     9.926441e-18
ADI      9.593299e-18
MSFT     8.873720e-18
VRSN     7.977628e-18
ANSS     7.503180e-18
SMCI     6.633186e-18
M


- Enforced param:
    - Target return ≥ 5%
    - Max weight = 10% per asset
    - Long-only constraint (no negative weights)
- As a result:
    - Assets like EA, AZN, KDP, AEP, and GILD hit the maximum allowed allocation of 10% — they are likely low-risk, high-return contributors.
    - Several other assets have near-zero weights (e.g., MRVL, MSTR) — the optimizer deems them suboptimal under your constraints.


Saving connstrained weights to the sql 


In [6]:
from sqlalchemy import create_engine

# --- Prepare final weights table ---
weights_df = optimal_weights_constrained.reset_index()
weights_df.columns = ['ticker', 'weight']
weights_df = weights_df[weights_df['weight'] > 0].sort_values(by='weight', ascending=False)

# --- Save to PostgreSQL ---
engine = create_engine('postgresql://akilfiros:@127.0.0.1:5432/postgres')
weights_df.to_sql("optimal_portfolio_weights", engine, if_exists='replace', index=False)

print("✅ Optimal weights saved to PostgreSQL table: optimal_portfolio_weights")

✅ Optimal weights saved to PostgreSQL table: optimal_portfolio_weights
